# 自然语言推理和数据集
:label:`sec_natural-language-inference-and-dataset`

在 :numref:`sec_sentiment` 中，我们讨论了情感分析的问题。
该任务旨在将单个文本序列分类到预定义的类别中，
例如一组情感极性。
然而，当需要决定一个句子是否可以从另一个句子推导出来，
或者通过识别语义等价的句子来消除冗余时，
只知道如何分类一个文本序列是不够的。
相反，我们需要能够对成对的文本序列进行推理。

## 自然语言推理

*自然语言推理* 研究一个 *假设* 是否可以从前提推导出来，其中两者都是文本序列。
换句话说，自然语言推理确定一对文本序列之间的逻辑关系。
这种关系通常分为三种类型：

* *蕴含*：可以从前提中推导出假设。
* *矛盾*：可以从前提中推导出假设的否定。
* *中立*：所有其他情况。

自然语言推理也被称为识别文本蕴含任务。
例如，以下配对将被标记为 *蕴含*，因为假设中的“表达感情”可以从前提中的“拥抱彼此”推导出来。

> 前提：两个女人正在拥抱彼此。

> 假设：两个女人正在表达感情。

以下是 *矛盾* 的例子，因为“运行代码示例”表明“没有睡觉”而不是“正在睡觉”。

> 前提：一个男人正在运行《深入深度学习》中的代码示例。

> 假设：这个男人正在睡觉。

第三个例子展示了 *中立* 关系，因为从“正在为我们表演”这一事实中既不能推导出“著名”也不能推导出“不著名”。

> 前提：音乐家们正在为我们表演。

> 假设：这些音乐家很著名。

自然语言推理一直是理解自然语言的核心话题。
它的应用范围广泛，从信息检索到开放领域问答。
为了研究这个问题，我们将从调查一个流行的自然语言推理基准数据集开始。

## 斯坦福自然语言推理（SNLI）数据集

[**斯坦福自然语言推理（SNLI）语料库**] 是一个包含超过500,000条带标签的英语句子对的数据集 :cite:`Bowman.Angeli.Potts.ea.2015`。
我们下载并将提取的SNLI数据集存储在路径 `../data/snli_1.0` 中。

In [1]:
import os
import re
import torch
from torch import nn
from d2l import torch as d2l

#@save
d2l.DATA_HUB['SNLI'] = (
    'https://nlp.stanford.edu/projects/snli/snli_1.0.zip',
    '9fcde07509c7e87ec61c640c1b2753d9041758e4')

data_dir = d2l.download_extract('SNLI')

### [**读取数据集**]

原始的SNLI数据集包含的信息比我们实验中实际需要的要丰富得多。因此，我们定义了一个函数`read_snli`来仅提取数据集的一部分，然后返回前提、假设及其标签的列表。

In [2]:
#@save
def read_snli(data_dir, is_train):
    """Read the SNLI dataset into premises, hypotheses, and labels."""
    def extract_text(s):
        # Remove information that will not be used by us
        s = re.sub('\\(', '', s)
        s = re.sub('\\)', '', s)
        # Substitute two or more consecutive whitespace with space
        s = re.sub('\\s{2,}', ' ', s)
        return s.strip()
    label_set = {'entailment': 0, 'contradiction': 1, 'neutral': 2}
    file_name = os.path.join(data_dir, 'snli_1.0_train.txt'
                             if is_train else 'snli_1.0_test.txt')
    with open(file_name, 'r') as f:
        rows = [row.split('\t') for row in f.readlines()[1:]]
    premises = [extract_text(row[1]) for row in rows if row[0] in label_set]
    hypotheses = [extract_text(row[2]) for row in rows if row[0] in label_set]
    labels = [label_set[row[0]] for row in rows if row[0] in label_set]
    return premises, hypotheses, labels

现在让我们[**打印前3对**]前提和假设，以及它们的标签（"0"、"1"和"2"分别对应"蕴含"、"矛盾"和"中立"）。

In [3]:
train_data = read_snli(data_dir, is_train=True)
for x0, x1, y in zip(train_data[0][:3], train_data[1][:3], train_data[2][:3]):
    print('premise:', x0)
    print('hypothesis:', x1)
    print('label:', y)

premise: A person on a horse jumps over a broken down airplane .
hypothesis: A person is training his horse for a competition .
label: 2
premise: A person on a horse jumps over a broken down airplane .
hypothesis: A person is at a diner , ordering an omelette .
label: 1
premise: A person on a horse jumps over a broken down airplane .
hypothesis: A person is outdoors , on a horse .
label: 0


训练集大约有550000对，
测试集大约有10000对。
以下显示
三个标签“entailment”、“contradiction”和“neutral”在
训练集和测试集中都是平衡的。

In [4]:
test_data = read_snli(data_dir, is_train=False)
for data in [train_data, test_data]:
    print([[row for row in data[2]].count(i) for i in range(3)])

[183416, 183187, 182764]
[3368, 3237, 3219]


### [**定义用于加载数据集的类**]

下面通过继承Gluon中的`Dataset`类来定义一个用于加载SNLI数据集的类。类构造函数中的参数`num_steps`指定了文本序列的长度，以便每个小批量的序列具有相同的形状。
换句话说，
较长序列中超过前`num_steps`个之后的标记将被裁剪，而较短的序列将附加特殊标记“&lt;pad&gt;”，直到其长度达到`num_steps`。
通过实现`__getitem__`函数，我们可以任意访问索引`idx`对应的前提、假设和标签。

In [5]:
#@save
class SNLIDataset(torch.utils.data.Dataset):
    """A customized dataset to load the SNLI dataset."""
    def __init__(self, dataset, num_steps, vocab=None):
        self.num_steps = num_steps
        all_premise_tokens = d2l.tokenize(dataset[0])
        all_hypothesis_tokens = d2l.tokenize(dataset[1])
        if vocab is None:
            self.vocab = d2l.Vocab(all_premise_tokens + all_hypothesis_tokens,
                                   min_freq=5, reserved_tokens=['<pad>'])
        else:
            self.vocab = vocab
        self.premises = self._pad(all_premise_tokens)
        self.hypotheses = self._pad(all_hypothesis_tokens)
        self.labels = torch.tensor(dataset[2])
        print('read ' + str(len(self.premises)) + ' examples')

    def _pad(self, lines):
        return torch.tensor([d2l.truncate_pad(
            self.vocab[line], self.num_steps, self.vocab['<pad>'])
                         for line in lines])

    def __getitem__(self, idx):
        return (self.premises[idx], self.hypotheses[idx]), self.labels[idx]

    def __len__(self):
        return len(self.premises)

### [**整合所有内容**]

现在我们可以调用`read_snli`函数和`SNLIDataset`类来下载SNLI数据集，并返回训练集和测试集的`DataLoader`实例，以及训练集的词汇表。
值得注意的是，我们必须使用从训练集中构建的词汇表
作为测试集的词汇表。
因此，测试集中出现的任何新词对于在训练集上训练的模型来说都是未知的。

In [6]:
#@save
def load_data_snli(batch_size, num_steps=50):
    """Download the SNLI dataset and return data iterators and vocabulary."""
    num_workers = d2l.get_dataloader_workers()
    data_dir = d2l.download_extract('SNLI')
    train_data = read_snli(data_dir, True)
    test_data = read_snli(data_dir, False)
    train_set = SNLIDataset(train_data, num_steps)
    test_set = SNLIDataset(test_data, num_steps, train_set.vocab)
    train_iter = torch.utils.data.DataLoader(train_set, batch_size,
                                             shuffle=True,
                                             num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(test_set, batch_size,
                                            shuffle=False,
                                            num_workers=num_workers)
    return train_iter, test_iter, train_set.vocab

在这里我们将批量大小设置为128，序列长度设置为50，
然后调用`load_data_snli`函数来获取数据迭代器和词汇表。
接着我们打印词汇表的大小。

In [7]:
train_iter, test_iter, vocab = load_data_snli(128, 50)
len(vocab)

read 549367 examples
read 9824 examples


18678

现在我们打印第一个小批量的形状。
与情感分析不同，
我们有两个输入 `X[0]` 和 `X[1]` 代表前提和假设的配对。

In [ ]:
for X, Y in train_iter:
    print(X[0].shape)
    print(X[1].shape)
    print(Y.shape)
    break

torch.Size([128, 50])
torch.Size([128, 50])
torch.Size([128])


## 摘要

* 自然语言推理研究从前提中能否推导出假设，其中前提是文本序列。
* 在自然语言推理中，前提和假设之间的关系包括蕴含、矛盾和中立。
* 斯坦福自然语言推理（SNLI）语料库是自然语言推理的一个流行基准数据集。

## 练习

1. 机器翻译长期以来都是基于输出翻译与真实翻译之间的表面$n$-gram匹配来进行评估的。你能设计一种使用自然语言推理来评估机器翻译结果的方法吗？
1. 我们如何调整超参数以减少词汇量？

[讨论](https://discuss.d2l.ai/t/1388)